In [280]:
import numpy as np
import networkx as nx
import nltk
from math import sqrt
from sklearn.metrics.pairwise import cosine_similarity

In [36]:
from functools import partial

class Infix(object):
    def __init__(self, func):
        self.func = func
    def __or__(self, other):
        return self.func(other)
    def __ror__(self, other):
        return Infix(partial(self.func, other))
    def __call__(self, v1, v2):
        return self.func(v1, v2)

In [12]:
def load():
    ppp = nltk.data.load('../../Downloads/ppp.txt', encoding='utf8')
    words_p = nltk.tokenize.wordpunct_tokenize(ppp)[130:]
    alw = nltk.data.load('../../Downloads/alw.txt', encoding='utf8')
    words_a = nltk.tokenize.wordpunct_tokenize(alw)[143:]
    words = words_a + words_p
    return words

In [460]:
max_nodes = 100
def process(KB, embeddings, sentence):
    def next_lvl(sentence):
        @Infix
        def to(a, b):
            x = embeddings[sentence[b]] - embeddings[sentence[a]]
            return x
        
        nl = []
        for index in range(len(sentence[:-1])):
            if KB.has_edge(sentence[index], sentence[index+1]):
                nl.append(KB[sentence[index]][sentence[index+1]]['node'])
            else:
                if KB.number_of_nodes()+1 < max_nodes:
                    KB.add_node(KB.number_of_nodes(), exp=.5)
                    KB.add_edge(sentence[index], sentence[index+1],
                               node=KB.number_of_nodes())
                    embeddings[KB.number_of_nodes()] = index |to| (index+1)
                else:
                    x = index |to| (index+1)
                    similarity = cosine_similarity(embeddings, [x])
                    nearest = np.argmax(similarity)
                    exp = KB.node[nearest]['exp']
                    embeddings[nearest] = embeddings[nearest]*exp + x*(1-exp)
                    KB.node[nearest]['exp'] += (1-exp)**3
                    if 1-similarity.max() > 1-exp:
                        nl.append(nearest)
                    
                    if not KB.has_edge(sentence[index], sentence[index+1]):
                        KB.add_edge(sentence[index], sentence[index+1], 
                                    node=nearest)
        return nl
    l_all = [sentence]
    l_next = sentence
    while l_next:
        print(l_next)
        l_next = next_lvl(l_next)
        l_all += [l_next]
    return KB, embeddings, l_all

In [364]:
words = load()

In [445]:
embeddings = np.zeros((100, 100))
for i in range(10):
    embeddings[i] += np.random.random(size=100)
KB = nx.DiGraph()
KB.add_nodes_from(np.arange(10), exp=.5)

In [461]:
KB, embeddings, sentence = process(KB, embeddings, [i for i in range(10,25)])

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
[22, 24, 25, 27, 28, 29, 31, 32, 33, 34, 35, 36, 37, 38]
[40, 41, 43, 44, 44, 46, 47, 48, 49, 50, 51, 52, 53]
[61, 63, 64, 65, 66, 67, 67, 67, 56, 57, 67, 67]
[69, 70, 71, 72, 73, 67, 67, 74, 67, 75, 67]
[81, 82, 83, 84, 85, 67, 86, 87, 78, 79]
[98, 99, 93]
